---

# 3.4 Newton's method and variants
---

Let 

$$f \in C^2[a,b].$$ 

That is, $f$ is a **twice-continuously differentiable** function over $[a,b]$, which means that the **first** and **second** derivatives of $f$ **exist** and are **continuous** on the open interval $(a,b)$. **Newton's method** is defined as:

$$ x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}, \quad k = 0, 1, 2, \ldots.$$

This is the fixed point iteration using the function $g(x) = x - f(x)\big/f'(x)$.

We have already seen above that this iteration can converge very fast. This is why Newton's method is one of the most popular methods for root finding.

## Formulating Newton's method

Suppose that $f(x^*) = 0$ and that we are at the iterate $x_k$. By the **Taylor Series Theorem**, we have

$$
f(x^*) = f(x_k) + f'(x_k)(x^* - x_k) + \frac{f''(\xi)}{2}(x^* - x_k)^2,
$$

for some point $\xi$ between $x^*$ and $x_k$. If $x_k$ is already fairly close to $x^*$, then $(x^* - x_k)^2$ will be very small, so we have

$$0 \approx f(x_k) + f'(x_k)(x^* - x_k).$$

Solving for $x^*$, we obtain

$$x^* \approx x_k - \frac{f(x_k)}{f'(x_k)}.$$

Therefore, it makes sense to define our next iterate $x_{k+1}$ using this approximation.

## Another formulation

Another way to obtain Newton's method is as follows. Consider the **first-order (linear) approximation** of $f$ around the point $x_k$:

$$f(x) \approx f(x_k) + f'(x_k)(x - x_k), \quad \text{for all $x \approx x_k$}.$$

Suppose that $x_k$ is close to $x^*$, and that $f(x^*) = 0$. Then

$$f(x^*) \approx f(x_k) + f'(x_k)(x^* - x_k),$$

which implies that

$$x^* \approx x_k - \frac{f(x_k)}{f'(x_k)}.$$

Therefore, our next iterate should be 

$$x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}.$$

---

## An implementation of Newton's Method: `plotnewton`

In [ ]:
using Plots, Printf

function plotnewton(f, fp, x0, a, b; tol=1e-6, maxiter=10)
    
    plt = plot(f, a, b, color=:blue, label=:none)
    hline!([0], color=:black, label=:none)
    
    k = 0
    x = x0
    fx = f(x)
    xold, xnew = x, Inf
    @printf "%4s %22s %22s\n" "k" "x" "f(x)"
    while abs(xnew - xold) > tol*(1 + abs(xold)) && k < maxiter
        k += 1
        xnew = x - fx/fp(x)
        
        # Plotting stuff
        plot!([x, x], [0, fx], ls=:dot, c=:black, label=:none)
        plot!([xnew, x], [0, fx], c=:red, label=:none)
        scatter!([x], [fx], c=:blue, label=:none)
        
        xold = x
        x = xnew
        fx = f(x)
        @printf "%4d %22.15e %22.15e\n" k x fx
    end
    flush(stdout)
    display(plt)
    
    return x
    
end

---

## Example

Let's see how Newton's method works on $f(x) = \sin(x)$.

In [ ]:
x = plotnewton(sin, cos, 4.3, 0, 2π)

In [ ]:
π

---

## Example

Let's use the above code to plot Newton's method using 

$$f(x) = 2\cosh(x/4) - x,$$

which has two roots,

$$x_1^* \approx 2.3575510538774020 \qquad \text{and} \qquad x_2^* \approx 8.5071995707130270.$$

Which root do you converge to starting at $x_0 = 2, 4, 8, 10$?

In [ ]:
xbad = 4asinh(2.)

In [ ]:
x = plotnewton(
    x -> 2cosh(x/4) - x,  # f(x)
    x -> .5sinh(x/4) - 1, # f'(x)
    4.0, 1, 9)

---

## Example (The Babylonian method for computing $\sqrt{a}$)

Let $f(x) = x^2 - a$. Newton's method gives us the iteration:

$$ x_{k+1} = x_k - \frac{x_k^2 - a}{2x_k} = \frac{1}{2}\left(x_k + \frac{a}{x_k}\right).$$

This is known as the [Babylonian method](http://en.wikipedia.org/wiki/Methods_of_computing_square_roots#Babylonian_method) for computing square roots. This method only requires addition and division, and converges rapidly.

In [ ]:
# Let's do this using BigFloat

setprecision(256)
a = big"10."
x = a
@printf "%4s %10s\n" "k" "|xk - √a|"
for k = 1:8
    x = (x + a/x)/2
    @printf "%4d %10.1e\n" k abs(x - sqrt(a))
end

In [ ]:
x

In [ ]:
sqrt(10.0)

---

## Speed of Convergence

If $x_k \to x^*$, we can measure the speed of the convergence as follows.

- **Linear convergence** means there is a constant $0 < \rho < 1$ such that

  $$\left|x_{k+1}-x^*\right| \leq \rho \left|x_k - x^*\right|, \quad \text{for all $k$ sufficiently large};$$
  
  that is,
  
  $$ \lim_{k \to \infty} \frac{\left|x_{k+1}-x^*\right|}{\left|x_k - x^*\right|} = \rho < 1.$$
  

- **Superlinear convergence** means there is a sequence $\rho_k \to 0$ such that

  $$\left|x_{k+1}-x^*\right| \leq \rho_k \left|x_k - x^*\right|, \quad \text{for all $k$ sufficiently large};$$

  that is,
  
  $$ \lim_{k \to \infty} \frac{\left|x_{k+1}-x^*\right|}{\left|x_k - x^*\right|} = 0.$$
  

- **Quadratic convergence** means there is a constant $M$ such that

  $$\left|x_{k+1}-x^*\right| \leq M \left|x_k - x^*\right|^2, \quad \text{for all $k$ sufficiently large};$$
  
  that is,
  
  $$ \lim_{k \to \infty} \frac{\left|x_{k+1}-x^*\right|}{\left|x_k - x^*\right|^2} = M < \infty.$$


Note that **quadratic convergence** is an example of **superlinear convergence** with $\rho_k = M \left|x_k - x^*\right|$.

---

> ## Theorem (Quadratic convergence of Newton's Method):
Let $f \in C^2[a,b]$. If $f$ has a root $x^* \in (a,b)$ such that $f'(x^*) \neq 0$, then there is a $\delta > 0$ such that Newton's method **converges quadratically** to $x^*$ from any $x_0 \in [x^*-\delta, x^*+\delta]$.

## Proof:

Since:

- $f \in C^2[a,b]$ 
- $x^* \in (a,b)$
- $f'(x^*) \neq 0$

there are positive constants $\delta_1$, $\varepsilon$, and $M$ such that

- $\left|f'(x)\right| \geq \varepsilon$ 
- $\left|f''(x)\right| \leq M$

for all $x \in [x^*-\delta_1, x^*+\delta_1] \subset (a,b)$.

Suppose $x_k \in [x^*-\delta_1, x^*+\delta_1]$. Then, there is a $\xi_k$ between $x^*$ and $x_k$ such that

$$ f(x^*) = f(x_k) + f'(x_k) (x^* - x_k) + \frac{f''(\xi_k)}{2} (x^* - x_k)^2.$$

Using the fact that $f(x^*) = 0$ and $f'(x_k) \neq 0$, we can rearrange this equation to get

$$ -\frac{f(x_k)}{f'(x_k)} - x^* + x_k =  \frac{f''(\xi_k)}{2f'(x_k)} (x^* - x_k)^2.$$

Since $x_{k+1} = x_k - f(x_k)\big/f'(x_k)$, we have 

$$x^* - x_{k+1} = -\frac{f''(\xi_k)}{2f'(x_k)} (x^* - x_k)^2.$$

Thus,

$$\left|x^* - x_{k+1}\right| = \left|\frac{f''(\xi_k)}{2f'(x_k)}\right| \left|x^* - x_k\right|^2 \leq \frac{M}{2\varepsilon} \left|x^* - x_k\right|^2,$$

so if $x_k \to x^*$, then the **convergence will be quadratic**.

We just need to find $\delta > 0$ so that if $x_0 \in [x^* - \delta, x^* + \delta]$, then $x_k \to x^*$.

Let $\lambda \in (0, 1)$ and let

$$\delta = \min\left\{\frac{2\varepsilon\lambda}{M}, \delta_1\right\}.$$

Suppose that $x_k \in [x^* - \delta, x^* + \delta]$. Then

$$
\begin{split}
\left|x^* - x_{k+1}\right| 
&\leq \frac{M}{2\varepsilon} \left|x^* - x_k\right|^2 \\
&\leq \frac{M}{2\varepsilon} \delta \left|x^* - x_k\right| \\
&\leq \lambda \left|x^* - x_k\right| \\
&< \delta,
\end{split}
$$

so $x_{k+1} \in [x^* - \delta, x^* + \delta]$ as well. Thus, if $x_0 \in [x^* - \delta, x^* + \delta]$, we have $x_k \in [x^* - \delta, x^* + \delta]$ for $k = 0, 1, 2, \ldots$. 

Moreover,

$$0 \leq \left|x^* - x_k\right| \leq \lambda \left|x^* - x_{k-1}\right| \leq \lambda^2 \left|x^* - x_{k-2}\right| \leq \cdots \leq \lambda^k \left|x^* - x_{0}\right|.$$

Since $\lambda \in (0,1)$, we have $\lambda^k \left|x^* - x_{0}\right| \to 0$ as $k \to \infty$. Thus, if $x_0 \in [x^* - \delta, x^* + \delta]$ then $x_k$ converges to $x^*$ quadratically. $\blacksquare$

---

## Pros and cons of Newton's method

Pros:

1. **Fast to converge:** Newton's method enjoys quadratic convergence near the root when $f'(x^*) \neq 0$.

2. **Generalizes to multiple variables:** Let $\mathbf{F} \colon \mathbb{R}^n \to \mathbb{R}^n$. Newton's method for solving 
   
   $$\mathbf{F}(\mathbf{x}) = \mathbf{0}$$ 
   
   (i.e., $n$ nonlinear equations with $n$ unknowns) is

   $$ \mathbf{x}_{k+1} = \mathbf{x}_k - \mathbf{J}(\mathbf{x}_k)^{-1} \mathbf{F}(\mathbf{x}_k),$$

   where $\mathbf{J}(\mathbf{x})$ is the $n \times n$ **Jacobian** of $\mathbf{F}$:
   
   $$
   \mathbf{J}(\mathbf{x}) = 
   \begin{bmatrix}
   \frac{\partial F_1}{\partial x_1}& \cdots &
   \frac{\partial F_1}{\partial x_n}\\
   \vdots & \ddots & \vdots \\
   \frac{\partial F_n}{\partial x_1}& \cdots &
   \frac{\partial F_n}{\partial x_n}\\
   \end{bmatrix}$$


Cons:

1. **Requires the derivative:** We must give Newton's method both the function $f$ and its derivative $f'$. This may not always be possible or easy.

2. **Need to start close to $x^*$:** Newton's method is a **local method**. When $x_0$ is far from $x^*$, Newton's method may not converge to $x^*$, or may require many iterations before quadratic convergence begins.


---

## Secant method

Sometimes it is not possible to evaluate the derivative $f'$:

- $f'$ is unknown or difficult to obtain
- evaluating $f'$ takes too much time

Instead, we can use the **secant approximation** of the derivative. When $x_k \approx x_{k-1}$, we have

$$ f'(x_k) \approx \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}}.$$

Plugging this approximation into the formula for Newton's method, we get:

$$x_{k+1} = x_k - \frac{f(x_k)(x_k - x_{k-1})}{f(x_k) - f(x_{k-1})}$$

The secant method is an example of a [quasi-Newton](http://en.wikipedia.org/wiki/Quasi-Newton_method) method since we are replacing $f'$ with an approximation of $f'$.

When $f'(x^*) \neq 0$, the secant method will converge **superlinearly**, so it may not be as fast as Newton's method.

---

## An implementation of the secant method: `plotsecant`

In [ ]:
using Plots, Printf

function plotsecant(f, x0, x1, a, b; tol=1e-6, maxiter=10, doplot=true)
    
    if doplot
        plt = plot(f, a, b, color=:blue, label=:none)
        hline!([0], color=:black, label=:none)
    end
    
    k = 0
    xprev = x0
    x = x1
    fxprev = f(xprev)
    fx = f(x)
    
    if doplot
        plot!([xprev, xprev], [0, fxprev], ls=:dot, c=:black, label=:none)
        scatter!([xprev], [fxprev], c=:blue, label=:none)
    end
    
    @printf "%4s %22s %22s\n" "k" "x" "f(x)"
    while abs(x - xprev) > tol*(1 + abs(x)) && k < maxiter
        k += 1
        xnew = x - fx*(x - xprev)/(fx - fxprev)
        
        # Plotting stuff
        if doplot
            plot!([x, x], [0, fx], ls=:dot, c=:black, label=:none)
            plot!([xnew, xprev, x], [0, fxprev, fx], c=:red, label=:none)
            scatter!([xprev], [fxprev], c=:blue, label=:none)
        end
        
        xprev = x
        x = xnew
        fxprev = f(xprev)
        fx = f(x)
        @printf "%4d %22.15e %22.15e\n" k x fx
    end
    flush(stdout)
    if doplot
        display(plt)
    end
    
    return x
end

---

## Example

Let's see how the secant method works on $f(x) = \sin(x)$.

In [ ]:
x = plotsecant(sin, 1.0, 4.3, 0, 2π, maxiter=100)

In [ ]:
abs(x - pi)

---

## Example

Recall the example of finding the value of $y_0$ such that the solution $y(t)$ of the ordinary differential equation 

$$
\frac{dy}{dt} = t - 5 + \frac{y^2}{100}
$$

with initial value $y(0) = y_0$ satisfies $y(10) = 20$.

In [ ]:
using OrdinaryDiffEq

F(y, p, t) = t - 5 + y^2/100

function foo(y0)
    tspan = (0.0, 10.0)
    prob = ODEProblem(F, y0, tspan)
    sol = solve(prob, Tsit5())
    return sol(10.0)
end

In [ ]:
foo(10.0)

In [ ]:
foo(15.0)

In [ ]:
x = plotsecant(y0 -> foo(y0) - 20, 10.0, 15.0, 10, 15, tol=1e-10)

In [ ]:
foo(x)

---

## The case of a multiple root

When $f'(x^*) = 0$, we are no longer guaranteed to obtain superlinear convergence of the secant method, nor quadratic convergence of Newton's method. In this case, both methods will be merely **linearly convergent**.

Let's try these methods on the function

$$f(x) = x^2$$

for which $x^* = 0$ is a root of **multiplicity** 2.

In [ ]:
x = plotnewton(x -> x^2, x -> 2x, 4.0, -1, 5, maxiter=100)

In [ ]:
x = plotsecant(x -> x^2, 4.0, 3.0, -1, 5, maxiter=100)

---